# Analyse des Résultats des Simulations du Modèle SEIR Multi-Agent

Ce notebook a pour objectif de lire les résultats de multiples réplications de simulations du modèle SEIR, de calculer les moyennes des états S, E, I, R à chaque pas de temps, et de visualiser ces dynamiques moyennes.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob # Pour trouver facilement les fichiers CSV

# Style pour les graphiques (optionnel, mais améliore le visuel)
plt.style.use('ggplot')

## 1. Configuration et Paramètres

Définissez ici le chemin vers le dossier contenant les fichiers CSV des résultats et le préfixe des fichiers si nécessaire.

In [ ]:
# Chemin vers le dossier contenant les fichiers CSV
# IMPORTANT : Modifiez cette variable pour qu'elle pointe vers le dossier où vos fichiers CSV sont sauvegardés
DOSSIER_RESULTATS = './' # Exemple: './resultats_simulations/' ou le répertoire courant

# Préfixe commun des fichiers de résultats (si applicable, sinon laisser vide ou adapter le glob)
# Par exemple, si vos fichiers sont nommés "toy_model_repl_1_seed0.csv", "toy_model_repl_2_seed1.csv", etc.
PREFIXE_FICHIER = 'toy_model_repl_'
SUFFIXE_FICHIER = '.csv'

# Nombre total de réplications attendu
NOMBRE_REPLICATIONS = 100 # Doit correspondre au nombre de fichiers CSV générés

# Nombre d'itérations par simulation
NB_ITERATIONS_SIMU = 730 # Doit correspondre au nombre de lignes de données par CSV (hors en-tête)

## 2. Lecture et Chargement des Données de Toutes les Réplications

Nous allons maintenant lire tous les fichiers CSV correspondant à nos réplications.

In [ ]:
# Construire le motif de recherche pour les fichiers CSV
# S'assure que le dossier se termine par un slash s'il est spécifié et non vide
if DOSSIER_RESULTATS and not DOSSIER_RESULTATS.endswith('/'):
    DOSSIER_RESULTATS += '/'

motif_recherche = f"{DOSSIER_RESULTATS}{PREFIXE_FICHIER}*{SUFFIXE_FICHIER}"
print(f"Motif de recherche des fichiers : {motif_recherche}")

tous_les_fichiers_csv = glob.glob(motif_recherche)

# Trier les fichiers pour s'assurer d'un ordre cohérent si nécessaire (optionnel)
# tous_les_fichiers_csv.sort() 

if not tous_les_fichiers_csv:
    print(f"ERREUR : Aucun fichier CSV trouvé dans '{DOSSIER_RESULTATS}' avec le préfixe '{PREFIXE_FICHIER}'. Vérifiez le chemin et le préfixe.")
else:
    print(f"Nombre de fichiers CSV trouvés : {len(tous_les_fichiers_csv)}")
    if len(tous_les_fichiers_csv) != NOMBRE_REPLICATIONS:
        print(f"ATTENTION : {len(tous_les_fichiers_csv)} fichiers trouvés, mais {NOMBRE_REPLICATIONS} étaient attendues.")

# Liste pour stocker les DataFrames de chaque réplication
liste_dataframes_replications = []

for i, nom_fichier in enumerate(tous_les_fichiers_csv):
    try:
        df_repl = pd.read_csv(nom_fichier)
        # Vérification basique de la structure
        if 'iteration' in df_repl.columns and len(df_repl) == (NB_ITERATIONS_SIMU + 1): # +1 pour la ligne d'itération 0
             # Ajouter une colonne pour identifier la réplication
            df_repl['replication_id'] = i + 1 # ou extraire l'ID du nom de fichier si pertinent
            liste_dataframes_replications.append(df_repl)
        else:
            print(f"ATTENTION: Le fichier {nom_fichier} ne semble pas avoir le bon format (colonnes manquantes ou nombre d'itérations incorrect). Il sera ignoré.")
            print(f"  Colonnes trouvées: {df_repl.columns.tolist()}, Lignes trouvées: {len(df_repl)}")
            
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier {nom_fichier}: {e}")

if liste_dataframes_replications:
    print(f"\n{len(liste_dataframes_replications)} DataFrames de réplication chargés avec succès.")
    # Afficher un aperçu du premier DataFrame chargé (optionnel)
    # print("\nAperçu du premier DataFrame chargé (premières lignes) :")
    # print(liste_dataframes_replications[0].head())
else:
    print("\nERREUR : Aucun DataFrame n'a pu être chargé. Vérifiez les fichiers CSV et les messages d'erreur précédents.")


## 3. Agrégation des Données

Si les DataFrames ont été chargés, nous les combinons en un seul grand DataFrame, puis nous calculons les moyennes pour chaque état (S, E, I, R) à chaque pas de temps (itération) sur l'ensemble des réplications.

In [ ]:
if liste_dataframes_replications:
    # Concaténer tous les DataFrames en un seul
    df_complet = pd.concat(liste_dataframes_replications, ignore_index=True)
    
    print("\nAperçu du DataFrame combiné (premières et dernières lignes) :")
    print(df_complet.head())
    print("...")
    print(df_complet.tail())
    
    # Calculer les moyennes pour S, E, I, R pour chaque itération
    # Le groupby sur 'iteration' va regrouper toutes les lignes ayant le même numéro d'itération (provenant des différentes réplications)
    # Ensuite, .mean() calcule la moyenne pour chaque colonne numérique (S, E, I, R) pour ces groupes.
    df_moyennes = df_complet.groupby('iteration')[['S', 'E', 'I', 'R']].mean()
    
    # Optionnel: calculer aussi l'écart-type ou d'autres statistiques
    df_std = df_complet.groupby('iteration')[['S', 'E', 'I', 'R']].std()
    df_min = df_complet.groupby('iteration')[['S', 'E', 'I', 'R']].min()
    df_max = df_complet.groupby('iteration')[['S', 'E', 'I', 'R']].max()

    print("\nTableau des moyennes par itération (premières lignes) :")
    print(df_moyennes.head())
else:
    print("\nPas de données à agréger car aucun DataFrame n'a été chargé.")
    df_moyennes = pd.DataFrame() # Créer un DataFrame vide pour éviter les erreurs suivantes

## 4. Visualisation des Résultats Moyens

Nous allons maintenant tracer les courbes moyennes de S, E, I, et R en fonction du temps (itérations).

In [ ]:
if not df_moyennes.empty:
    plt.figure(figsize=(12, 8))
    
    plt.plot(df_moyennes.index, df_moyennes['S'], label='Susceptibles (S) - Moyenne')
    plt.plot(df_moyennes.index, df_moyennes['E'], label='Exposés (E) - Moyenne')
    plt.plot(df_moyennes.index, df_moyennes['I'], label='Infectieux (I) - Moyenne')
    plt.plot(df_moyennes.index, df_moyennes['R'], label='Rétablis (R) - Moyenne')
    
    # Optionnel: Afficher les intervalles de confiance ou min/max
    # plt.fill_between(df_moyennes.index, df_min['I'], df_max['I'], color='red', alpha=0.1, label='Min/Max Infectieux')

    plt.title('Dynamique Moyenne du Modèle SEIR sur {} Réplications'.format(len(liste_dataframes_replications) if liste_dataframes_replications else 'N/A'))
    plt.xlabel('Temps (jours/itérations)')
    plt.ylabel("Nombre d'individus")
    plt.legend()
    plt.grid(True)
    plt.show()
else:
    print("Impossible de générer le graphique car les données moyennes sont vides.")


## 5. (Optionnel) Visualisation des Écarts-Types ou Intervalles de Confiance

Si vous avez calculé les écarts-types, vous pouvez les visualiser ici pour montrer la variabilité entre les simulations.

In [ ]:
# Exemple pour tracer les moyennes avec une bande pour l'écart-type (si df_std a été calculé)
if not df_moyennes.empty and 'df_std' in locals() and not df_std.empty :
    plt.figure(figsize=(12, 8))
    
    for state in ['S', 'E', 'I', 'R']:
        mean_values = df_moyennes[state]
        std_values = df_std[state]
        
        plt.plot(df_moyennes.index, mean_values, label=f'{state} - Moyenne')
        plt.fill_between(df_moyennes.index, mean_values - std_values, mean_values + std_values, alpha=0.2, label=f'{state} - Écart-type')

    plt.title('Dynamique Moyenne du Modèle SEIR avec Écart-Type sur {} Réplications'.format(len(liste_dataframes_replications)))
    plt.xlabel('Temps (jours/itérations)')
    plt.ylabel("Nombre d'individus")
    # Pour éviter trop de labels si on trace S,E,I,R et leurs std séparément, on peut gérer la légende plus finement
    # plt.legend() # Peut devenir chargé, ajuster au besoin
    handles, labels = plt.gca().get_legend_handles_labels()
    # Filtrer pour n'avoir que les labels de moyenne ou créer des labels personnalisés
    # Par exemple, prendre un label sur deux si chaque plot + fill_between ajoute un label
    unique_labels = {}
    for i, label in enumerate(labels):
        if label not in unique_labels:
            unique_labels[label] = handles[i]
    plt.legend(unique_labels.values(), unique_labels.keys())

    plt.grid(True)
    plt.show()
else:
    print("Graphique avec écart-type non généré (données manquantes).")
